# Search engineeee

In [1]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import re

# Crawl synopsis for trying to do search

In [2]:
def scrabbing_search_engine(soup):
    anime_info = []
    #title
    title = str(soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string)
    anime_info.append(title)
    #synopsis
    synopsis = str(soup.find("p", attrs = {"itemprop": "description"}).text)
    anime_info.append(synopsis)
    
    return anime_info

In [4]:
attrs = ["animeTitle","animeDescription","animeUrl"]
list_of_anime = []

# take info
for page in tqdm(range(1,50)):
    folder = "./Folder_with_page/page"+str(page)
    for anime in os.listdir(folder):
        with open(folder + "/" + anime, "r",  encoding='utf-8') as fp:
            soup = BeautifulSoup(fp, "html.parser")
        anime_info = scrabbing_search_engine(soup)
        anime_info.append(soup.find("meta",  property="og:url")["content"])
        list_of_anime.append(anime_info)
        
# Creating the DataFrame
df = pd.DataFrame(list_of_anime, columns = attrs)

# Creating the tsv file
# Index True, in this way = ID of the anime
df.to_csv('prova_search_engine.tsv', index = False, sep = '\t')

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [03:05<00:00,  3.79s/it]


# Creating the Search Engine 

First of all lets gather all the documents in one list.
* documents: lists of document. Each document correspond to an anime description.

In [5]:
documents = []
for doc in df["animeDescription"]:
    documents.append(doc)

## Clean the documents

Now let's cleaning all the documents. This step is colled preprocessing. We follow this order:
- 1) expand contraction form + Normalization (capital lower words)
- 2) remove number from text. We want only text string 
- 3) Tokanize. We divide the string in words.
- 4) removing stopwords
- 5) removing punctuation
- 6) removing some other words or non-text string
- 7) stemming 

Let's inspect the document to see what we can delete and what no:
- for example: "Philosopher's Stone—a powerful" << This is dash
- but "bio-mechanical engineering" << This is hyphen

We decide to keep the hypen and remove the dash

Also we encounter a lot of contraction form: using wikipedia https://en.wikipedia.org/wiki/Wikipedia:List_of_English_contractions
we store them in a dictionary and restore the long form.

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [7]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he shall",
"he'll've": "he shall have",
"he's": "he has",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I shall",
"i'll've": "I shall have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it shall",
"it'll've": "it shall have",
"it's": "it has",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she shall",
"she'll've": "she shall have",
"she's": "she has",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that has",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there has",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they shall",
"they'll've": "they shall have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall",
"what'll've": "what shall have",
"what're": "what are",
"what's": "what has",
"what've": "what have",
"when's": "when has",
"when've": "when have",
"where'd": "where did",
"where's": "where has",
"where've": "where have",
"who'll": "who shall",
"who'll've": "who shall have",
"who's": "who has",
"who've": "who have",
"why's": "why has",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you shall",
"you'll've": "you shall have",
"you're": "you are",
"you've": "you have"
}

def replace_words(d, contractions):
    for key, value in contractions.items():
        d = d.replace(key, value)
    return d

In [8]:
def pre_processing(documents):
    stop = stopwords.words("english")
    snowball_stemmer = SnowballStemmer("english")
    lmtzr = WordNetLemmatizer()
    remove = ["Written", "MAL", "Rewrite"]+["'s"]+["``",'"',"'","“","''"]

    
    # removing contraction + Normalization
    document_tmp = replace_words(documents.lower(), contractions)
    # remove number ( ordinal number too) 
    document_tmp = re.sub(r'[0-9]+(?:st|nd|rd|th)', '', document_tmp)
    # remove dash "—"
    document_tmp = document_tmp.replace('—',' ')
    # Tokenizing + Normalization
    document_tmp =  word_tokenize(document_tmp) 
    # removing stopwords
    document_tmp = [ word for word in document_tmp if word not in stop]
    # removing punctuation
    document_tmp = [ word for word in document_tmp if word not in string.punctuation]
    # removing "Written MAL Rewrite" and other stuff
    document_tmp = [ word for word in document_tmp if word not in remove]
    # lemmatize
    document_tmp = [ lmtzr.lemmatize(word) for word in document_tmp]
    # stemming 
    document_tmp = [ snowball_stemmer.stem(word) for word in document_tmp]

    
    return document_tmp

- documents_clean: is a list of list of the documents cleaned. Each list contain the tokenize cleaning document text.

In [9]:
# cleaning the documents
documents_clean = []
for d in documents:
    documents_clean.append(pre_processing(d))

Let's view how a document is processed:

In [10]:
documents_clean[0][:10]

['horrif',
 'alchemi',
 'experi',
 'go',
 'wrong',
 'elric',
 'household',
 'brother',
 'edward',
 'alphons']

# Creating vocabulary

In [11]:
import itertools
import numpy as np

I will create a list of each unique word among the all documents

In [12]:
# the list of all words
word_list = list(set(list(itertools.chain.from_iterable(documents_clean))))

Creating a dictionary that maps each word to an integer

In [13]:
vocabolary = dict(zip(word_list, range(len(word_list))))

In [14]:
# view the vocabolary
count = 0
for key, mapped_int in vocabolary.items():
    count +=1
    print(key,"-->",mapped_int)
    if count == 10: break

reliabl --> 0
note --> 1
prank --> 2
qiao --> 3
theft --> 4
ill-assort --> 5
plaster --> 6
sight --> 7
self-discoveri --> 8
re-liv --> 9


## Search Engine v1 Conjunctive query

For this type of search engine we need only to have a search engine based on the query appear or not in each documents. 


* ### Prepare the mapped document

To do this we will create an array of documents of each len(document_j) in which thare are converted the word into integer based on the vocabolary

We will use numpy array for time optimitation.

In [34]:
# function that map document text to integer

def word_to_int(document, vocabolary):
    int_doc = np.zeros(len(document), dtype = np.int64)
    # iterating over the document that has len(d)<<len(vocabolary)
    # change the value of the document, otherwise remain zero
    for i, word in enumerate(document):
        # vocabolary[word] is the mapping function that return an integer i.e the index
        int_doc[i] = vocabolary[word]
        
    return np.sort(int_doc)

* documents_mapped: is a list of list that have the words mapped

In [39]:
documents_mapped = []
for d in documents_clean:
    documents_mapped.append(word_to_int(d,vocabolary))

In [62]:
documents_mapped[0]

array([  118,   356,   437,   496,  1031,  1031,  1353,  1515,  1602,
        1730,  1796,  2040,  2049,  2122,  2129,  2211,  2230,  2233,
        2307,  2716,  2813,  2886,  3080,  3116,  3395,  3414,  3581,
        3754,  4352,  4739,  4772,  5031,  5031,  5031,  5031,  5554,
        5629,  5690,  5780,  6040,  6165,  6190,  6794,  6940,  7043,
        7120,  7368,  7372,  7824,  7882,  7964,  8258,  8468,  8483,
        8665,  8862,  8862,  8932,  9132,  9361,  9511, 10137, 10293,
       10313, 10582, 10734, 10734, 10734, 10742, 10890, 10905, 11053,
       11119, 11119, 11131, 11290, 11368, 11471, 11624, 11817, 11905,
       11980, 11980, 12290, 12569, 12590, 12971, 13063, 13164, 13176,
       13191, 13569, 13655, 13782, 13931, 14183, 14442, 14599, 14791,
       14920, 15212, 15212, 15237, 15237, 15413, 15518, 15665, 15665,
       15665, 15725, 15832, 15834, 16056], dtype=int64)

* ### Inverted Index v1

In [42]:
from collections import defaultdict  

In [43]:
Inverted_index = defaultdict(list)

To compute the Inverted Index we iterating over each document. Every time we encounter a word (that is now a integer) we insert in the dictionary the id of the documents, which is the row index in documents_mapped or in the dataframe.

In [44]:
for i,d in enumerate(documents_mapped):
    for word in set(d):
        Inverted_index[word].append(i)

Saving the Inverted Index in memory

In [394]:
import json

file = open("Inverted_index.json", "w")
json.dump(Inverted_index, file)
file.close()

* ### Searching

In [46]:
# input query
query_text = input()

 alchemist alchemy


In [47]:
# pre-processing the query
query_clean = pre_processing(query_text)
query_int = word_to_int(query_clean, vocabolary)

In [48]:
query_int

array([ 4772, 11119], dtype=int64)

We create a list named index that store a list of list, each one is the output from the inverted_index corrisponding to a query element. Than we intersect for obtain the documents that match ALL the query elements.

In [54]:
index = []
for query in query_int:
    index.append(set(Inverted_index[query]))

index = list(index[0].intersection(*index))
print(index)

[0, 393, 525, 167]


In [55]:
df.iloc[index,:]

,animeTitle,animeDescription,animeUrl
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/5114/Fullmetal_A...
393,Fullmetal Alchemist,"Edward Elric, a young, brilliant alchemist, ha...",https://myanimelist.net/anime/121/Fullmetal_Al...
525,Fullmetal Alchemist: Brotherhood Specials,Amazing secrets and startling facts are expose...,https://myanimelist.net/anime/6421/Fullmetal_A...
167,Baccano!,"During the early 1930s in Chicago, the transco...",https://myanimelist.net/anime/2251/Baccano


# Searching Engine v2 Ranking

Now we want for our Inverted_index two element:

* $\text{tf}_{i,j}$: occurancy of term $j$ in document $i$
* $\text{idf}_{j}$: Inverse Document Frequency of term $j$

Creating the $\text{tf}_{i,j}$ matrix:

In [76]:
# document: the document in exam
# n_words: total number of words in vocabolary

def tfij(documents,n_words):
    tf = np.zeros((len(documents), n_words), dtype = np.int64)
    
    for i,d in enumerate(documents):
        for word in d:
            tf[i][word] += 1
        
    return tf

Define:

* n_words = total number of words in vocabolary
* n = number of documents

In [79]:
# tf_ij

n_words = len(vocabolary)
n = len(documents)

tf = tfij(documents_mapped, n_words)

Creating the $\text{idf}_{j}$ array:
* I need the $n_j$: number of documents containing term j

In [80]:
nj = np.zeros(n_words, dtype = np.int64)

for d in documents_mapped:
    for word in set(d):
        nj[word] += 1

In [81]:
# creating idf_j

idf = np.zeros(n_words, dtype = np.float64)

idf = np.log(n/nj) / np.log(n)
    

In [84]:
tfIdf = np.multiply(tf,idf)

In [85]:
tfIdf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

* ### Inverted indexi v2

In [42]:
from collections import defaultdict  

In [171]:
Inverted_index = defaultdict(list)

To compute the Inverted Index we iterating over each document. Every time we encounter a word (that is now a integer) we insert in the dictionary the __tupla__ of _id_ of the documents and the _tfIdf_.

In [172]:
for i,d in enumerate(documents_mapped):
    for word in set(d):
        Inverted_index[word].append((i,tfIdf[i,word]))

Saving the Inverted Index in memory

In [394]:
import json

file = open("Inverted_index.json", "w")
json.dump(Inverted_index, file)
file.close()

* ### Searching

In [216]:
# input query
query_text = input()

 alchemist alchemy mother


In [217]:
query_clean = pre_processing(query_text)
query_int = word_to_int(query_clean, vocabolary)

q_tf = np.zeros(n_words, dtype=np.int64)
for value in query_int:
    q_tf[value] += 1

We define the cosine similarity as:
$$
\begin{equation}
\cos(q,d^i) =  \frac{\sum_{j=1}^d q_j*d_{ij}}{||d^i||}
\end{equation}
$$
we elimate the $||d^i||$ because is equal to all the documents and I'm interested in ranking.
$$
\begin{equation}
score(q,d^i) =  \cos(q,d^i)*||q||
\end{equation}
$$
Recall: $d^i = [\text{tfIdf}_{i1}, \text{tfIdf}_{i2}, \ldots]$ 

In [218]:
def score(q,d):
    q_norm = np.linalg.norm(q)
    d_norm = np.linalg.norm(d)
    
    score = np.dot(q,d)*(q_norm/d_norm)
    return score

In [219]:
query_int

array([ 4772, 11119, 13176], dtype=int64)

Create the list of each match 

In [220]:
match_list = []
for query in query_int:
    match_list.append(Inverted_index[query])

In [252]:
match = True
pointer = np.zeros(len(query_int), dtype = np.int64)
scores = []
while(match):
    # take the element in the pointer 
    idd = []
    d_tfidf = []
    for i in range(len(query_int)):
        idd_tmp, d_tmp = match_list[i][pointer[i]]
        idd.append(idd_tmp)
        d_tfidf.append(d_tmp)
    
    if len(set(idd)) > 1: # they are not ugual
        min_value = min(idd)
        min_index = idd.index(min_value)
        pointer[min_index] += 1
        if pointer[min_index] > 40:
            print("miao")
            match = False
    else:
        scores.append(score(d_tfidf,tfIdf[idd[0]],q_tf))
        pointer += 1

miao


In [253]:
scores

[0.5833176535311918, 1.0333026264161131]

In [243]:
def score(d_match, d, q):
    q_norm = np.linalg.norm(q)
    d_norm = np.linalg.norm(d)
    
    score = sum(d_match)*(q_norm)/(d_norm)
    return score

In [254]:
import heapq as hp

In [255]:
max_score = hp.nlargest(10, list(scores))
index = []
for i in max_score:
    index.append(int(np.where(scores == i)[0]))

In [256]:
index

[1, 0]

In [257]:
topk = df.iloc[index].copy()
topk["scores"] = max_score

In [258]:
topk

,animeTitle,animeDescription,animeUrl,scores
1,Gintama: The Final,New Gintama movie.,https://myanimelist.net/anime/39486/Gintama__T...,1.033303
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/5114/Fullmetal_A...,0.583318
